# Manual Confocal-to-Anatomy Prematch GUI

Interactive GUI for manually prematching confocal stacks to 2-photon anatomy.
Saves translation (x, y) and rotation values to the processing log for each session.

**Usage:**
- Run standalone: Execute all cells to prematch all animals/sessions
- From pipeline: Call `run_manual_prematch_gui(animals, cfg)` programmatically

**Requirements:**
- `fireantsGH` conda environment
- `ipywidgets`, `matplotlib`, `scipy`, `numpy`, `tifffile`

In [ ]:
from pathlib import Path
import json
from datetime import datetime, timezone
from typing import List, Dict, Optional, Tuple

import numpy as np
import tifffile
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg
from scipy.ndimage import rotate as nd_rotate, shift as nd_shift
import ipywidgets as widgets
from IPython.display import display, clear_output

from social_imaging_scripts.metadata.config import load_project_config, resolve_output_path
from social_imaging_scripts.metadata.loader import load_animals
from social_imaging_scripts.metadata.models import AnimalMetadata, AnatomySession
from social_imaging_scripts.pipeline.processing_log import (
    build_processing_log_path,
    load_processing_log,
    save_processing_log,
    AnimalProcessingLog,
)

print("Imports successful.")

## Helper Functions

In [ ]:
def load_mip(stack_path: Path) -> np.ndarray:
    """Load a 3D stack and return XY maximum intensity projection."""
    stack = tifffile.imread(str(stack_path))
    if stack.ndim == 2:
        return stack
    elif stack.ndim == 3:
        return np.max(stack, axis=0)
    else:
        raise ValueError(f"Expected 2D or 3D stack, got {stack.ndim}D")


def load_pixel_size_from_metadata(metadata_path: Path, stack_type: str) -> Tuple[float, float]:
    """Load XY pixel size from preprocessing metadata.
    
    Args:
        metadata_path: Path to preprocessing metadata JSON
        stack_type: 'anatomy' or 'confocal'
    
    Returns:
        (pixel_size_y, pixel_size_x) in micrometers
    """
    try:
        meta = json.loads(metadata_path.read_text())
        
        if stack_type == "anatomy":
            # 2p anatomy: pixel_size_xy_um is [x, y]
            px, py = meta.get("pixel_size_xy_um", [1.0, 1.0])
            return (float(py), float(px))
        
        elif stack_type == "confocal":
            # Confocal: voxel_size_um is [x, y, z]
            # XY pixel sizes are typically the same; Z is different (e.g., 3.0 µm)
            voxel = meta.get("voxel_size_um", [1.0, 1.0, 1.0])
            # voxel[0] = x, voxel[1] = y, voxel[2] = z
            return (float(voxel[1]), float(voxel[0]))  # Return (y, x)
        
        else:
            raise ValueError(f"Unknown stack_type: {stack_type}")
    
    except Exception as e:
        print(f"Warning: Could not load pixel size from {metadata_path}: {e}")
        return (1.0, 1.0)


def rescale_image(img: np.ndarray, scale_y: float, scale_x: float) -> np.ndarray:
    """Rescale image by given factors using scipy zoom.
    
    Args:
        img: Input 2D image
        scale_y: Scaling factor for Y dimension
        scale_x: Scaling factor for X dimension
    
    Returns:
        Rescaled image
    """
    from scipy.ndimage import zoom
    return zoom(img, (scale_y, scale_x), order=1)


def normalize_image(img: np.ndarray, percentile: float = 99.5) -> np.ndarray:
    """Normalize image to [0, 1] using percentile scaling."""
    img = img.astype(np.float32)
    vmin, vmax = np.percentile(img, [0.5, percentile])
    if vmax > vmin:
        img = (img - vmin) / (vmax - vmin)
    img = np.clip(img, 0, 1)
    return img


def apply_transform(
    img: np.ndarray, x_shift: float, y_shift: float, rotation_deg: float
) -> np.ndarray:
    """Apply 2D rotation and translation to an image."""
    # Rotate first (around center)
    if abs(rotation_deg) > 0.01:
        img = nd_rotate(img, rotation_deg, reshape=False, order=1, mode='constant', cval=0)
    # Then translate
    if abs(x_shift) > 0.01 or abs(y_shift) > 0.01:
        img = nd_shift(img, shift=(y_shift, x_shift), order=1, mode='constant', cval=0)
    return img


def create_overlay(
    fixed: np.ndarray, moving: np.ndarray, alpha: float = 0.5
) -> np.ndarray:
    """Create RGB overlay: fixed=magenta, moving=green."""
    fixed = normalize_image(fixed)
    moving = normalize_image(moving)
    
    overlay = np.zeros((*fixed.shape, 3), dtype=np.float32)
    overlay[..., 0] = fixed * 0.8  # Red channel (for magenta)
    overlay[..., 1] = moving * alpha + fixed * (1 - alpha) * 0.5  # Green
    overlay[..., 2] = fixed * 0.8  # Blue channel (for magenta)
    overlay = np.clip(overlay, 0, 1)
    return overlay


def get_existing_prematch(
    animal_id: str, session_id: str, cfg
) -> Optional[Dict[str, float]]:
    """Load existing manual prematch from processing log if present."""
    try:
        log_path = build_processing_log_path(
            cfg.processing_log, animal_id, base_dir=Path(cfg.output_base_dir)
        )
        if not log_path.exists():
            return None
        
        log = load_processing_log(log_path)
        stage = log.stages.get("confocal_to_anatomy_registration")
        if not stage:
            return None
        
        manual_prematch = stage.parameters.get("manual_prematch", {})
        session_data = manual_prematch.get(session_id)
        return session_data
    except Exception as e:
        print(f"Warning: Could not load existing prematch: {e}")
        return None


def save_prematch_to_log(
    animal_id: str,
    session_id: str,
    x_shift: float,
    y_shift: float,
    rotation_deg: float,
    cfg,
) -> None:
    """Save manual prematch values to processing log."""
    log_path = build_processing_log_path(
        cfg.processing_log, animal_id, base_dir=Path(cfg.output_base_dir)
    )
    
    # Load or create log
    if log_path.exists():
        log = load_processing_log(log_path)
    else:
        log = AnimalProcessingLog(animal_id=animal_id)
    
    # Ensure stage exists
    stage = log.ensure_stage("confocal_to_anatomy_registration")
    
    # Get or create manual_prematch dict
    if "manual_prematch" not in stage.parameters:
        stage.parameters["manual_prematch"] = {}
    
    # Save session data
    stage.parameters["manual_prematch"][session_id] = {
        "translation_x_px": float(x_shift),
        "translation_y_px": float(y_shift),
        "rotation_deg": float(rotation_deg),
        "created_at": datetime.now(tz=timezone.utc).isoformat(),
        "method": "manual_gui",
    }
    
    log.touch()
    save_processing_log(log, log_path)
    print(f"✓ Saved prematch for {animal_id}/{session_id} to {log_path}")


print("Helper functions defined.")

## Collect Sessions Needing Prematch

In [ ]:
def collect_sessions_needing_prematch(
    animals: List[AnimalMetadata], cfg, force: bool = False
) -> List[Tuple[AnimalMetadata, AnatomySession, AnatomySession]]:
    """Collect all (animal, confocal_session, anatomy_session) tuples needing prematch.
    
    Args:
        animals: List of animals to check
        cfg: Project configuration
        force: If True, include all sessions even if prematch exists
    
    Returns:
        List of (animal, confocal_session, anatomy_session) tuples
    """
    sessions_to_match = []
    
    for animal in animals:
        # Find 2p anatomy session (stack_type: two_photon)
        anatomy_session = None
        for session in animal.sessions:
            if session.session_type == "anatomy_stack":
                stack_type = getattr(session.session_data, "stack_type", None)
                if stack_type == "two_photon":
                    anatomy_session = session
                    break
        
        if anatomy_session is None:
            print(f"⚠ {animal.animal_id}: No 2p anatomy session found, skipping")
            continue
        
        # Check anatomy preprocessing output exists
        anatomy_root = resolve_output_path(
            animal.animal_id,
            cfg.anatomy_preprocessing.root_subdir,
            cfg=cfg,
        )
        anatomy_stack_path = anatomy_root / cfg.anatomy_preprocessing.stack_filename_template.format(
            animal_id=animal.animal_id,
            session_id=anatomy_session.session_id,
        )
        
        if not anatomy_stack_path.exists():
            print(f"⚠ {animal.animal_id}: Anatomy stack not found at {anatomy_stack_path}, skipping")
            continue
        
        # Find confocal sessions (stack_type: confocal)
        for session in animal.sessions:
            if session.session_type != "anatomy_stack":
                continue
            
            # Check if this is a confocal session (not two_photon)
            stack_type = getattr(session.session_data, "stack_type", None)
            if stack_type != "confocal":
                continue
            
            # Check if confocal preprocessing output exists
            confocal_root = resolve_output_path(
                animal.animal_id,
                cfg.confocal_preprocessing.root_subdir,
                session.session_id,
                cfg=cfg,
            )
            
            # Look for reference channel (gcamp by default)
            ref_channel = cfg.confocal_to_anatomy_registration.reference_channel_name.lower()
            channel_path = confocal_root / cfg.confocal_preprocessing.channel_filename_template.format(
                channel=ref_channel
            )
            
            if not channel_path.exists():
                # Try finding any channel
                channel_files = list(confocal_root.glob("channel_*.tif"))
                if not channel_files:
                    print(f"⚠ {animal.animal_id}/{session.session_id}: No confocal channels found, skipping")
                    continue
                channel_path = channel_files[0]
            
            # Check if prematch already exists
            existing = get_existing_prematch(animal.animal_id, session.session_id, cfg)
            if existing and not force:
                print(f"✓ {animal.animal_id}/{session.session_id}: Prematch exists (x={existing.get('translation_x_px', 0):.1f}, y={existing.get('translation_y_px', 0):.1f}, rot={existing.get('rotation_deg', 0):.1f}°)")
                continue
            
            sessions_to_match.append((animal, session, anatomy_session))
    
    return sessions_to_match


print("Session collection function defined.")

## Interactive GUI Class

In [ ]:
class ManualPrematchGUI:
    """Interactive GUI for manual confocal-to-anatomy prematching."""
    
    def __init__(
        self,
        animal: AnimalMetadata,
        confocal_session: AnatomySession,
        anatomy_session: AnatomySession,
        cfg,
    ):
        self.animal = animal
        self.confocal_session = confocal_session
        self.anatomy_session = anatomy_session
        self.cfg = cfg
        
        # Load images with pixel size correction
        self._load_images()
        
        # Load existing prematch if available
        existing = get_existing_prematch(animal.animal_id, confocal_session.session_id, cfg)
        init_x = existing.get("translation_x_px", 0.0) if existing else 0.0
        init_y = existing.get("translation_y_px", 0.0) if existing else 0.0
        init_rot = existing.get("rotation_deg", 0.0) if existing else 0.0
        
        # Create widgets
        self._create_widgets(init_x, init_y, init_rot)
        
        # Initial render
        self._update_display(None)
    
    def _load_images(self):
        """Load anatomy (fixed) and confocal (moving) MIPs with pixel size correction."""
        # Load anatomy MIP and metadata
        anatomy_root = resolve_output_path(
            self.animal.animal_id,
            self.cfg.anatomy_preprocessing.root_subdir,
            cfg=self.cfg,
        )
        anatomy_stack_path = anatomy_root / self.cfg.anatomy_preprocessing.stack_filename_template.format(
            animal_id=self.animal.animal_id,
            session_id=self.anatomy_session.session_id,
        )
        anatomy_metadata_path = anatomy_root / self.cfg.anatomy_preprocessing.metadata_filename_template.format(
            animal_id=self.animal.animal_id,
            session_id=self.anatomy_session.session_id,
        )
        self.fixed_mip = load_mip(anatomy_stack_path)
        self.fixed_pixel_size = load_pixel_size_from_metadata(anatomy_metadata_path, "anatomy")
        
        # Load confocal MIP and metadata
        confocal_root = resolve_output_path(
            self.animal.animal_id,
            self.cfg.confocal_preprocessing.root_subdir,
            self.confocal_session.session_id,
            cfg=self.cfg,
        )
        ref_channel = self.cfg.confocal_to_anatomy_registration.reference_channel_name.lower()
        channel_path = confocal_root / self.cfg.confocal_preprocessing.channel_filename_template.format(
            channel=ref_channel
        )
        if not channel_path.exists():
            channel_path = list(confocal_root.glob("channel_*.tif"))[0]
        
        confocal_metadata_path = confocal_root / self.cfg.confocal_preprocessing.metadata_filename_template.format(
            session_id=self.confocal_session.session_id,
            animal_id=self.animal.animal_id,
        )
        self.moving_mip = load_mip(channel_path)
        self.moving_pixel_size = load_pixel_size_from_metadata(confocal_metadata_path, "confocal")
        
        # Calculate scaling factors to match confocal to anatomy pixel size
        # scale = confocal_pixel / anatomy_pixel (if confocal has larger pixels, we need to upsample)
        self.scale_y = self.moving_pixel_size[0] / self.fixed_pixel_size[0]
        self.scale_x = self.moving_pixel_size[1] / self.fixed_pixel_size[1]
        
        print(f"Pixel sizes - Anatomy: {self.fixed_pixel_size} µm, Confocal: {self.moving_pixel_size} µm")
        print(f"Scaling confocal by (Y={self.scale_y:.3f}, X={self.scale_x:.3f}) to match anatomy resolution")
        
        # Rescale confocal to match anatomy pixel size
        self.moving_mip_rescaled = rescale_image(self.moving_mip, self.scale_y, self.scale_x)
        
        # Pad to same size
        max_h = max(self.fixed_mip.shape[0], self.moving_mip_rescaled.shape[0])
        max_w = max(self.fixed_mip.shape[1], self.moving_mip_rescaled.shape[1])
        
        def pad_to_size(img, h, w):
            pad_h = (h - img.shape[0]) // 2
            pad_w = (w - img.shape[1]) // 2
            return np.pad(
                img,
                ((pad_h, h - img.shape[0] - pad_h), (pad_w, w - img.shape[1] - pad_w)),
                mode='constant',
                constant_values=0,
            )
        
        self.fixed_mip = pad_to_size(self.fixed_mip, max_h, max_w)
        self.moving_mip_rescaled = pad_to_size(self.moving_mip_rescaled, max_h, max_w)
    
    def _create_widgets(self, init_x, init_y, init_rot):
        """Create GUI widgets."""
        # Determine slider ranges based on image size
        max_shift = max(self.fixed_mip.shape) // 2
        
        # Sliders
        self.x_slider = widgets.FloatSlider(
            value=init_x,
            min=-max_shift,
            max=max_shift,
            step=1.0,
            description='X shift (px):',
            continuous_update=False,
            layout=widgets.Layout(width='500px'),
        )
        
        self.y_slider = widgets.FloatSlider(
            value=init_y,
            min=-max_shift,
            max=max_shift,
            step=1.0,
            description='Y shift (px):',
            continuous_update=False,
            layout=widgets.Layout(width='500px'),
        )
        
        self.rot_slider = widgets.FloatSlider(
            value=init_rot,
            min=-180,
            max=180,
            step=1.0,
            description='Rotation (°):',
            continuous_update=False,
            layout=widgets.Layout(width='500px'),
        )
        
        # Alpha blend slider
        self.alpha_slider = widgets.FloatSlider(
            value=0.5,
            min=0.0,
            max=1.0,
            step=0.05,
            description='Moving alpha:',
            continuous_update=False,
            layout=widgets.Layout(width='500px'),
        )
        
        # Buttons
        self.save_button = widgets.Button(
            description='Save to Metadata',
            button_style='success',
            icon='check',
        )
        # Note: callback will be set by run_manual_prematch_gui
        
        self.skip_button = widgets.Button(
            description='Skip',
            button_style='warning',
            icon='forward',
        )
        # Note: callback will be set by run_manual_prematch_gui
        
        self.reset_button = widgets.Button(
            description='Reset',
            button_style='info',
            icon='refresh',
        )
        self.reset_button.on_click(self._on_reset)
        
        # Info text
        self.info_label = widgets.HTML(
            value=f"<b>Animal:</b> {self.animal.animal_id} | <b>Confocal:</b> {self.confocal_session.session_id} | <b>Anatomy:</b> {self.anatomy_session.session_id}<br>"
                  f"<b>Fixed (magenta):</b> 2p anatomy ({self.fixed_pixel_size[1]:.3f}×{self.fixed_pixel_size[0]:.3f} µm/px) | "
                  f"<b>Moving (green):</b> confocal ({self.moving_pixel_size[1]:.3f}×{self.moving_pixel_size[0]:.3f} µm/px, rescaled)"
        )
        
        self.status_label = widgets.HTML(value="")
        
        # Output widget for matplotlib figure
        self.output = widgets.Output()
        
        # Observe slider changes
        self.x_slider.observe(self._update_display, names='value')
        self.y_slider.observe(self._update_display, names='value')
        self.rot_slider.observe(self._update_display, names='value')
        self.alpha_slider.observe(self._update_display, names='value')
    
    def _update_display(self, change):
        """Update the overlay display."""
        x_shift = self.x_slider.value
        y_shift = self.y_slider.value
        rotation = self.rot_slider.value
        alpha = self.alpha_slider.value
        
        # Apply transform to rescaled moving image
        moving_transformed = apply_transform(self.moving_mip_rescaled, x_shift, y_shift, rotation)
        
        # Create overlay
        overlay = create_overlay(self.fixed_mip, moving_transformed, alpha=alpha)
        
        # Render
        with self.output:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(overlay)
            ax.set_title(
                f"Overlay: x={x_shift:.1f}px, y={y_shift:.1f}px, rot={rotation:.1f}°",
                fontsize=12,
            )
            ax.axis('off')
            plt.tight_layout()
            plt.show()
    
    def _on_save(self, button):
        """Save prematch to processing log."""
        save_prematch_to_log(
            animal_id=self.animal.animal_id,
            session_id=self.confocal_session.session_id,
            x_shift=self.x_slider.value,
            y_shift=self.y_slider.value,
            rotation_deg=self.rot_slider.value,
            cfg=self.cfg,
        )
        self.status_label.value = "<span style='color:green;font-weight:bold;'>✓ Saved!</span>"
    
    def _on_skip(self, button):
        """Skip this session."""
        self.status_label.value = "<span style='color:orange;font-weight:bold;'>⊘ Skipped</span>"
    
    def _on_reset(self, button):
        """Reset sliders to zero."""
        self.x_slider.value = 0.0
        self.y_slider.value = 0.0
        self.rot_slider.value = 0.0
        self.status_label.value = ""
    
    def display(self):
        """Display the GUI."""
        display(
            widgets.VBox([
                self.info_label,
                self.output,
                self.x_slider,
                self.y_slider,
                self.rot_slider,
                self.alpha_slider,
                widgets.HBox([self.save_button, self.skip_button, self.reset_button]),
                self.status_label,
            ])
        )


print("ManualPrematchGUI class defined.")

## Main Execution Function

In [ ]:
def run_manual_prematch_gui(
    animals: Optional[List[AnimalMetadata]] = None,
    cfg = None,
    force: bool = False,
) -> Dict[str, int]:
    """Run manual prematch GUI for all sessions needing prematching.
    
    Args:
        animals: List of animals to process (default: all from metadata)
        cfg: Project config (default: load from defaults)
        force: If True, process all sessions even if prematch exists
    
    Returns:
        Dictionary with statistics: {'saved': N, 'skipped': M, 'total': T}
    """
    if cfg is None:
        cfg = load_project_config()
    
    if animals is None:
        animals = list(load_animals(base_dir=Path.cwd().parent / "metadata" / "animals").animals)
    
    # Collect sessions
    sessions = collect_sessions_needing_prematch(animals, cfg, force=force)
    
    if not sessions:
        print("\n✓ No sessions need prematching!")
        return {'saved': 0, 'skipped': 0, 'total': 0}
    
    print(f"\n📋 Found {len(sessions)} session(s) needing prematch.\n")
    
    # Create state object to track progress
    state = {
        'current_idx': 0,
        'sessions': sessions,
        'stats': {'saved': 0, 'skipped': 0, 'total': len(sessions)},
        'cfg': cfg,
    }
    
    # Display first GUI
    _show_next_session(state)
    
    return state['stats']


def _show_next_session(state):
    """Show the next session GUI or complete the process."""
    if state['current_idx'] >= len(state['sessions']):
        # All done!
        clear_output(wait=True)
        print("\n" + "="*80)
        print("📊 Manual Prematch Complete")
        print("="*80)
        print(f"Total sessions:  {state['stats']['total']}")
        print(f"Saved:           {state['stats']['saved']}")
        print(f"Skipped:         {state['stats']['skipped']}")
        print("="*80)
        return
    
    idx = state['current_idx']
    animal, confocal_session, anatomy_session = state['sessions'][idx]
    
    clear_output(wait=True)
    print(f"\n{'='*80}")
    print(f"Session {idx + 1}/{len(state['sessions'])}: {animal.animal_id} / {confocal_session.session_id}")
    print(f"{'='*80}\n")
    
    # Create GUI with callbacks
    gui = ManualPrematchGUI(animal, confocal_session, anatomy_session, state['cfg'])
    
    # Override button callbacks to advance to next session
    original_on_save = gui._on_save
    original_on_skip = gui._on_skip
    
    def on_save_and_next(button):
        original_on_save(button)
        state['stats']['saved'] += 1
        state['current_idx'] += 1
        # Small delay to show the "Saved!" message
        import time
        time.sleep(0.5)
        _show_next_session(state)
    
    def on_skip_and_next(button):
        original_on_skip(button)
        state['stats']['skipped'] += 1
        state['current_idx'] += 1
        _show_next_session(state)
    
    # Replace button callbacks
    gui.save_button.on_click(on_save_and_next)
    gui.skip_button.on_click(on_skip_and_next)
    
    gui.display()


print("Main execution function defined.")

## Standalone Execution

Run this cell to prematch all animals/sessions that need it.

**Note:** Processing logs are saved to `{output_base_dir}/metadata/processed/{animal_id}.yaml` (not in repo).

In [ ]:
# Load configuration and animals
cfg = load_project_config()
animals = list(load_animals(base_dir=Path.cwd().parent / "metadata" / "animals").animals)

print(f"Loaded {len(animals)} animal(s) from metadata.")

# Optional: Filter to specific animals
animals = [a for a in animals if a.animal_id in ["L331_f01", "L395_f06"]]

# Run GUI (set force=True to redo existing prematches)
stats = run_manual_prematch_gui(animals=animals, cfg=cfg, force=False)

## Transfer Manual Prematch Data

One-time script to transfer manual prematch data from repo logs to output logs.

In [ ]:
def transfer_prematch_data():
    """Transfer manual prematch data from repo logs to output logs."""
    
    cfg = load_project_config()
    repo_log_dir = Path.cwd().parent / "metadata" / "processed"
    output_log_dir = Path(cfg.output_base_dir) / "metadata" / "processed"
    
    print(f"Transferring prematch data:")
    print(f"  FROM: {repo_log_dir}")
    print(f"  TO:   {output_log_dir}\n")
    
    if not repo_log_dir.exists():
        print("No repo logs found to transfer.")
        return
    
    transferred = 0
    
    for repo_log_path in repo_log_dir.glob("*.yaml"):
        animal_id = repo_log_path.stem
        
        # Load repo log
        repo_log = load_processing_log(repo_log_path)
        
        # Check if it has manual prematch data
        stage = repo_log.stages.get("confocal_to_anatomy_registration")
        if not stage:
            continue
        
        manual_prematch = stage.parameters.get("manual_prematch", {})
        if not manual_prematch:
            continue
        
        print(f"📦 {animal_id}: Found {len(manual_prematch)} prematch session(s)")
        
        # Load or create output log
        output_log_path = output_log_dir / repo_log_path.name
        if output_log_path.exists():
            output_log = load_processing_log(output_log_path)
        else:
            output_log = AnimalProcessingLog(animal_id=animal_id)
        
        # Ensure stage exists in output log
        output_stage = output_log.ensure_stage("confocal_to_anatomy_registration")
        
        # Merge manual_prematch data
        if "manual_prematch" not in output_stage.parameters:
            output_stage.parameters["manual_prematch"] = {}
        
        for session_id, prematch_data in manual_prematch.items():
            output_stage.parameters["manual_prematch"][session_id] = prematch_data
            print(f"  ✓ {session_id}: x={prematch_data['translation_x_px']:.1f}, y={prematch_data['translation_y_px']:.1f}, rot={prematch_data['rotation_deg']:.1f}°")
        
        # Save output log
        output_log.touch()
        save_processing_log(output_log, output_log_path)
        transferred += 1
    
    print(f"\n✅ Transferred prematch data for {transferred} animal(s)")
    print(f"Output logs location: {output_log_dir}")


# Run transfer
transfer_prematch_data()

## Clean Up Old Repo Logs (Optional)

After confirming the transfer was successful, you can optionally remove the old logs from the repo.

In [ ]:
# Uncomment to delete old repo logs after verifying transfer
# import shutil
# repo_log_dir = Path.cwd().parent / "metadata" / "processed"
# if repo_log_dir.exists():
#     shutil.rmtree(repo_log_dir)
#     print(f"✓ Removed old repo logs from {repo_log_dir}")
# else:
#     print("No repo logs to clean up.")